<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/RNN_in_class_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RNN on "tiny-shakespear"

The following code trains an RNN on the [tiny-shakespear dataset](https://www.tensorflow.org/datasets/catalog/tiny_shakespeare).

In [1]:
#!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np

## Fetch the dataset.
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

## Convert the text to a sequence of integer indices representing characters.
chars = sorted(set(text))
char2idx = {u:i for i, u in enumerate(chars)}
idx2char = np.array(chars)
text_as_int = np.array([char2idx[c] for c in text])

# Create training examples and targets
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Define and create the RNN
vocab_size = len(chars)
embedding_dim = 256
rnn_units = 1024
model = Sequential([
    Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    SimpleRNN(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Train the model
model.compile(optimizer='adam', loss=loss)
history = model.fit(dataset, epochs=10)


Epoch 1/10
172/172 [==============================] - 27s 101ms/step - loss: 2.8490
Epoch 2/10
172/172 [==============================] - 17s 89ms/step - loss: 2.0390
Epoch 3/10
172/172 [==============================] - 18s 88ms/step - loss: 1.8431
Epoch 4/10
172/172 [==============================] - 17s 89ms/step - loss: 1.7085
Epoch 5/10
172/172 [==============================] - 19s 94ms/step - loss: 1.6169
Epoch 6/10
172/172 [==============================] - 17s 88ms/step - loss: 1.5523
Epoch 7/10
172/172 [==============================] - 17s 87ms/step - loss: 1.5043
Epoch 8/10
172/172 [==============================] - 17s 90ms/step - loss: 1.4673
Epoch 9/10
172/172 [==============================] - 18s 90ms/step - loss: 1.4377
Epoch 10/10
172/172 [==============================] - 18s 93ms/step - loss: 1.4121


In [7]:
# Generate some text

model.save_weights('./shakespeare_rnn_weights')  # Save the weights to the current directory

# Define the model with batch size of 1 for text generation
model_gen = Sequential([
    Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    SimpleRNN(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])

# Load the weights of the trained model
model_gen.load_weights(tf.train.latest_checkpoint("./"))

# Build the model
model_gen.build(tf.TensorShape([1, None]))


# Define the seed text and other parameters
seed_text = ""
num_generate = 1000  # Number of characters to generate
temperature = 1.0  # Higher value: more random, Lower value: more deterministic

# Handle empty string seed by initializing with a random character from the vocabulary
if not seed_text:
    seed_text = np.random.choice(chars)  # Choose a random character from chars

# Convert seed text to numerical representation
input_eval = [char2idx[s] for s in seed_text]
input_eval = tf.expand_dims(input_eval, 0)  # Add batch dimension

# Initialize the list to hold the generated text
text_generated = []

# Reset the states of the model
model_gen.reset_states()

# Use model_gen for text generation
for i in range(num_generate):
    predictions = model_gen(input_eval)
    predictions = tf.squeeze(predictions, 0)  # remove the batch dimension

    # Use a categorical distribution to sample the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Pass the predicted character as the next input to the model along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

# Combine the seed text with the generated text
generated_text = seed_text + ''.join(text_generated)
print(generated_text)




?

GRUMIO:
Not once again;
And lived, madam:
I loved mystand,
That land to reath to inveltire my leave foll carrain. O will an' calls to Rone!

Second Servingman:
Marching a gue offend
Thy justice, and all desper you
Of prove me come propesst freely.

BENVILINE:
It is the demation of
Tup mine, come fool the ling of retutt was the days is nother?

CLIFFORD:
We are not to art tappaintation.

CAMILLO:
Marcius here storm.

YORK:
I shall I do
you weak infains
Callageshal-sowinn of too.

PAULIGH:
Best on a happy said, mine ownet disnowift dast office. What, uncle may s,
So more men as we new warm
parte:
You know, keaven cansless thich.
Madam, say's by int. A neitness:
Poor signior off?
If now the rest,
As so desire 'twixt that hoted and this sing hereage.

POLIXENES:
When your hasse?
Up of my years corseowords: the kill's. would make no marriour will
Your earthy shall be many in plagues thine.
Thou hast said, heir die! sound not
by your stoprey,
Nay, it thank this sturs
Known fear'ly fault i

##Improvements:


1. Is the training slow or fast? If slow, how to improve it?
2. Try generating text using the model without starting with any context. What do you notice? Why did the text not meet your expectation? How can you improve it? Hints: (context window? tokenization?).  

